In [13]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection

In [14]:
apartments = pd.read_csv('resources/data/apartments_train.csv')
buildings = pd.read_csv('resources/data/buildings_train.csv')
nan_data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

In [21]:

distance = np.sqrt((37.6 - nan_data["longitude"])**2 + (55.75 - nan_data["latitude"])**2)
nan_data["distance"] = distance
data_train, data_valid = model_selection.train_test_split(nan_data, test_size=0.33, stratify=np.log(nan_data.price).round())

def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5


#input_x = test_data[["area_total", "distance", "rooms", "norm_const", "stories", "floor"]]
#in_dist = test_data[["district"]]

X_train = data_train[["area_total", "distance", "rooms", "constructed", "district"]]
y_train = np.log(data_train.loc[X_train.index].price)
X_valid = data_valid[["area_total", "distance", "rooms", "constructed", "district"]]
y_valid = np.log(data_valid.loc[X_valid.index].price)

dtrain = xgb.DMatrix(X_train, label = y_train)
dval = xgb.DMatrix(X_valid)
param = {'max_depth': 3, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}
num_round = 500

bst = xgb.train(param, dtrain, num_round)
preds_train = bst.predict(dtrain)
preds_valid = bst.predict(dval)


print(root_mean_squared_log_error(preds_train, y_train))
print(root_mean_squared_log_error(preds_valid, y_valid))


0.0123577778759877
0.012500789535784243


In [ ]:
t_apartments = pd.read_csv('resources/data/apartments_test.csv')
t_buildings = pd.read_csv('resources/data/buildings_test.csv')
test_data_a = pd.merge(t_apartments, t_buildings.set_index('id'), how='left', left_on='building_id', right_index=True)
X_test_nan_a = test_data_a.isna().any(axis=1)

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_data_a["id"]
submission.loc[~X_test_nan, 'price_prediction'] = np.exp(numpy_res) # Predict on non-nan entries
submission['price_prediction'].fillna(nn_data["price"].mean(), inplace=True)
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
submission.to_csv('xgb_submission.csv', index=False)

# Look at submitted csv
print('\nLine count of submission')
!wc -l sample_submission.csv